In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', 'CLIENTNUM'], axis=1)
cols = df.columns
num_data = list(df._get_numeric_data().columns)
cat_data = list(set(cols) - set(num_data))
num_data.remove('Dependent_count')
num_data.remove('Total_Relationship_Count')
num_data.remove('Months_Inactive_12_mon')
num_data.remove('Contacts_Count_12_mon')
cat_data.append('Dependent_count')
cat_data.append('Total_Relationship_Count')
cat_data.append('Months_Inactive_12_mon')
cat_data.append('Contacts_Count_12_mon')
num_data_plot = num_data
cat_data_plot = cat_data

df = df.loc[df['Card_Category'] != 'Platinum']
df = df.loc[df['Total_Ct_Chng_Q4_Q1'] <= 1.6]
df = df.loc[df['Months_Inactive_12_mon'] >= 1]
df = df.loc[df['Months_Inactive_12_mon'] < 6]
df = df.loc[df['Customer_Age'] <= 66]
df = df.loc[df['Total_Amt_Chng_Q4_Q1'] <= 1.5]
df['Credit_Limit'] = np.log10(df['Credit_Limit'])
fitted_data, fitted_lambda = stats.boxcox(df['Avg_Open_To_Buy'])
df['Avg_Open_To_Buy'] = fitted_data
df['Total_Trans_Amt'] = np.log10(df['Total_Trans_Amt'])
df['Avg_Utilization_Ratio'] = (df['Avg_Utilization_Ratio'])**(1/2)
#sns.histplot(df, x= 'Avg_Open_To_Buy', hue='Attrition_Flag', kde=True)
#for i in cols:
#    sns.histplot(df, x= i, hue='Attrition_Flag', kde=True)
#    plt.show()
#    df[i].value_counts()

In [ ]:
y = df['Attrition_Flag']
x = df.drop(['Attrition_Flag', 'Customer_Age', 'Gender', 'Marital_Status', 'Card_Category', 'Months_on_book'], axis=1)
col = x.columns
num_data = list(x._get_numeric_data().columns)
cat_data = list(set(col) - set(num_data))
x = pd.get_dummies(x)
y = y.replace(['Existing Customer', 'Attrited Customer'], [1, 0])

for i in num_data:
    pass
    sns.boxplot(data=df, x='Total_Relationship_Count', y= i, hue='Attrition_Flag')
    plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x = x.values
y = y.values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x_scaled, y, test_size = 0.1, random_state = 0)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#parameters = {'C':[0.01, 0.1, 1, 10, 100],
#                'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}

parameters = {'C':[0.01, 0.1, 1, 10, 100],
                'solver':('newton-cg', 'liblinear', 'sag', 'saga')}
logisReg = LogisticRegression()
clf = GridSearchCV(logisReg, parameters, scoring='f1')
clf.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

print(clf.best_params_)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

y_score = clf.decision_function(x_test)
n_classes = y.shape[1]
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(
    fpr,
    tpr,
    color="darkorange",
    lw=lw,
    label="ROC curve (area = %0.2f)" % roc_auc,
)
plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver operating characteristic example")
plt.legend(loc="lower right")
plt.show()